In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, AutoModelForSequenceClassification
import torch
from collections import Counter
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
data = pd.read_csv('../data/results/data.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    score = logits.softmax(dim=1).tolist()[0]
    return model.config.id2label[predicted_class_id], score